In [ ]:
import os
import sys
import yaml

import pandas as pd
import numpy as np

from scipy.ndimage import gaussian_filter

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
plt.style.use(["bmh", "../matplotlibrc"])
%matplotlib inline

sys.path.insert(0, os.getcwd() + "/../scripts")

import _helpers as h

In [ ]:
with open("../config.pypsaeur.yaml", 'r') as stream:
    peur_config = yaml.safe_load(stream)
    TECH_COLORS = peur_config["plotting"]["tech_colors"]
    NICE_NAMES = peur_config["plotting"]["nice_names"]

In [ ]:
# global settings
filter_sigma = 1
pcolormesh = False
colors = dict(H2="mediumvioletred", offwind="cornflowerblue", onwind="darkblue", wind="royalblue",
              solar="darkorange", battery="seagreen", transmission="firebrick")
ylim_upper_abs = dict(H2=800, battery=800, wind=1800, onwind=1800, offwind=1800, solar=1800, transmission=1200)
ylim_upper_rel = dict(H2=5, battery=5, wind=5, onwind=5, offwind=5, solar=5, transmission=3)
levels = [10, 25, 50, 75, 90, 99]

with open("../config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
def prepare_from_dataset(fn, carrier, sense):
    df = h.load_dataset(fn, carrier, sense)[carrier].unstack("epsilon").T
    df.index = df.index.astype(float)
    return df

In [ ]:
prepare_from_dataset("../results/dataset_high.csv", "wind", "min")

In [ ]:
pd.read_csv("../results/dataset_high.csv", index_col=0, header=list(range(8)))

In [ ]:
def prepare_from_surrogate(path, fidelity, carrier, sense, epsilons, size, rule='halton', fmt=3):
    uncertainties = config["uncertainties"]
    distribution = h.NamedJ(uncertainties)
    samples = distribution.sample(size, rule, fmt)

    predictions = {}
    for eps in epsilons:
        surrogate = h.NamedPoly.from_txt(f"{path}/polynomial-{fidelity}-{sense}-{carrier}-{eps}.txt")
        predictions[eps] = h.build_pce_prediction(surrogate, samples)

    opt_surrogate = h.NamedPoly.from_txt(f"{path}/polynomial-{fidelity}-min-cost-0.0.txt")
    predictions[0.0] = h.build_pce_prediction(opt_surrogate, samples)

    df = pd.concat(predictions, axis=0)
    df.index.set_names("epsilon", level=0, inplace=True)
    
    return df

In [ ]:
def plot_density(mi, ma, carrier, fn=None, relative=False):
    
    # -----------------
    # prepare data
    # -----------------
    
    # settings
    columns = np.arange(0, 0.08 + 0.001, 0.002)
    if relative:
        step = 0.01
        index = np.arange(0, ylim_upper_rel[carrier] + step, step)
        mi = mi.divide(mi.loc[0.00,:], axis=1)
        ma = ma.divide(ma.loc[0.00,:], axis=1)
    else: 
        step = 5
        index = np.arange(0, ylim_upper_abs[carrier] + step, step)

    # TODO probably can do without pre-initialising dataframe
    df = pd.DataFrame(0, index=index, columns=columns)

    mi_i = mi.reindex(index=columns).interpolate("quadratic", axis=0)
    ma_i = ma.reindex(index=columns).interpolate("quadratic", axis=0)

    def to_heatmap(x, mi_i, ma_i):
        s = pd.concat({"min": mi_i.loc[x.name, :],
                       "max": ma_i.loc[x.name, :]}, axis=1)
        df = pd.Series([len(s.loc[(s["min"]<i) & (s["max"]>i)]) / len(s) * 100 for i in x.index], index=x.index)
        return df

    df = df.apply(to_heatmap, axis=0, args=(mi_i, ma_i))
    df.columns = [c * 100 for c in df.columns]

    if filter_sigma:
        df.loc[:,:] = gaussian_filter(df.values, sigma=filter_sigma)

    # -----------------
    # plot figure
    # -----------------

    fig, ax = plt.subplots(figsize=(4.5,4))
    plt.box(False)

    cmap = ListedColormap(sns.light_palette(colors[carrier], len(levels)).as_hex())

    if not pcolormesh:
        plt.contourf(df.columns, df.index, df, cmap=cmap, levels=levels, extend='max')

        # colorbar
        cbaxes = inset_axes(ax, width="30%", height="3%", loc="upper left", borderpad=1)#bbox_to_anchor=(-.5,.1,.8,0.8), bbox_transform=ax.transAxes) 
        cbar = plt.colorbar(cax=cbaxes, orientation='horizontal')
        cbar.ax.tick_params(labelsize=9)
        cbar.set_label(label="quantiles [%]",size=9)

        cs = ax.contour(df.columns, df.index, df, levels=levels, colors='darkgray', linewidths=0.5, extend='max')
        ax.clabel(cs, inline=True, fmt='%1.0f')
    else:
        plt.pcolormesh(df.columns, df.index, df, cmap="Reds")

    ax.grid(False)
    ax.legend(frameon=False, loc='upper left', bbox_to_anchor=(0,1.07))

    ax.set_title(NICE_NAMES[carrier])
    unit = "TWkm" if carrier == "transmission" else "GW"
    if relative: unit = "-"
    ax.set_ylabel(f"capacity [{unit}]")
    ax.set_xlabel(r"$\varepsilon$ [%]")

    # padding
    
    if relative:
        ypad = 0.1
        xpad = 0.1
    else:
        ypad = 20
        xpad = 0.4
        
    ylim = ax.get_ylim()
    ax.set_ylim(ylim[0] - ypad, ylim[1] + ypad)
    xlim = ax.get_xlim()
    ax.set_xlim(xlim[0] - xpad, xlim[1] + xpad)

    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')
        
    plt.close(fig)

In [ ]:
for carrier in ylim_upper_abs.keys():
    mi = prepare_from_dataset(f"../results/dataset_low.csv", carrier, "min")
    ma = prepare_from_dataset(f"../results/dataset_low.csv", carrier, "max")
    plot_density(mi, ma, carrier, fn=f"graphics/neardensity/samp-low-{carrier}.pdf")
    plot_density(mi, ma, carrier, fn=f"graphics/neardensity/samp-rel-low-{carrier}.pdf", relative=True)

In [ ]:
epsilons = [0.01,0.02,0.04,0.06,0.08] 
nsamples = 5000

In [ ]:
for fidelity in ["low", "high"]:
    for carrier in ylim_upper_abs.keys():
        mis = prepare_from_surrogate("../results/pce", fidelity, carrier, "min", epsilons, nsamples)
        mas = prepare_from_surrogate("../results/pce", fidelity, carrier, "max", epsilons, nsamples)
        mi = mis[carrier].unstack("epsilon").T
        ma = mas[carrier].unstack("epsilon").T
        mi.index = mi.index.astype(float)
        ma.index = ma.index.astype(float)

        plot_density(mi, ma, carrier, fn=f"graphics/neardensity/surr-{fidelity}-{carrier}.pdf")
        #plot_density(mi, ma, carrier, fn=f"graphics/neardensity/surr-rel-{fidelity}-{carrier}.pdf", relative=True)